In [1]:
# import the required libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
from skimage import io
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import random

### Reading CSV file containing path of all images with mask label

In [2]:
brain_df = pd.read_csv('Brain_MRI/data_mask.csv')

In [3]:
# Get the dataframe containing MRIs which have masks associated with them.
brain_df_mask = brain_df[brain_df['mask'] == 1]
brain_df_mask.shape

(1373, 4)

In [4]:
brain_df_mask = brain_df_mask.drop('patient_id', axis=1)

In [5]:
brain_df_mask.head()

,image_path,mask_path,mask
445,TCGA_CS_5393_19990606/TCGA_CS_5393_19990606_5.tif,TCGA_CS_5393_19990606/TCGA_CS_5393_19990606_5_...,1
507,TCGA_HT_7680_19970202/TCGA_HT_7680_19970202_5.tif,TCGA_HT_7680_19970202/TCGA_HT_7680_19970202_5_...,1
551,TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_6.tif,TCGA_CS_4944_20010208/TCGA_CS_4944_20010208_6_...,1
555,TCGA_CS_5393_19990606/TCGA_CS_5393_19990606_6.tif,TCGA_CS_5393_19990606/TCGA_CS_5393_19990606_6_...,1
617,TCGA_HT_7680_19970202/TCGA_HT_7680_19970202_6.tif,TCGA_HT_7680_19970202/TCGA_HT_7680_19970202_6_...,1


### Creating Image Datagen

In [6]:
# split the data into train, val and test data

from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(brain_df_mask, test_size=0.15)
X_test, X_val = train_test_split(X_val, test_size=0.5)

In [7]:
# create a image generator
from keras_preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255.)

In [8]:
def datagen_func(df):
    # original image data generator
    image_generator=datagen.flow_from_dataframe(dataframe=df,
                                                directory='./Brain_MRI/',
                                                x_col="image_path",
                                                batch_size= 16,
                                                class_mode=None,
                                                target_size=(256,256),
                                                color_mode='rgb')
    # mask data generator
    mask_generator=datagen.flow_from_dataframe(dataframe=df,
                                               directory='./Brain_MRI/',
                                               x_col="mask_path",
                                               batch_size=16,
                                               class_mode=None,
                                               target_size=(256,256),
                                               color_mode='grayscale')
    
    return image_generator, mask_generator

In [9]:
# getting train images
timage_generator, tmask_generator = datagen_func(X_train)

Found 1167 validated image filenames.
Found 1167 validated image filenames.


In [10]:
# getting val images
vimage_generator, vmask_generator = datagen_func(X_val)

Found 103 validated image filenames.
Found 103 validated image filenames.


In [11]:
# getting test images
testimage_generator, testmask_generator = datagen_func(X_test)

Found 103 validated image filenames.
Found 103 validated image filenames.


In [12]:
# creating an iterator of images
def data_iterator(image_gen, mask_gen):
    for img, mask in zip(image_gen, mask_gen):
        yield img, mask

In [13]:
train_gen = data_iterator(timage_generator, tmask_generator)
valid_gen = data_iterator(vimage_generator, vmask_generator)
test_gen = data_iterator(testimage_generator, testmask_generator)

### Designing the Model

In [14]:
# function to get downsapling convolutional block and skip connection

def conv_block(inputs=None, n_filters=32, dropout_prob=0, max_pooling=True):

    conv = Conv2D(n_filters,
                  3,   
                  activation='relu',
                  padding='same',
                  kernel_initializer='he_normal')(inputs)
    
    conv = Conv2D(n_filters,
                  3,
                  activation='relu',
                  padding='same',
                  kernel_initializer='he_normal')(conv)
    
    if dropout_prob > 0:
        conv = Dropout(dropout_prob)(conv)
         
    if max_pooling:
        next_layer = MaxPooling2D(2,2)(conv)
        
    else:
        next_layer = conv
        
    skip_connection = conv
    
    return next_layer, skip_connection

In [15]:
# function to get Convolutional upsampling block
    
def upsampling_block(expansive_input, contractive_input, n_filters=32):
    
    up = Conv2DTranspose(
                 n_filters,
                 (3,3),
                 strides=(2,2),
                 padding='same')(expansive_input)
    
    merge = concatenate([up, contractive_input], axis=3)
    
    conv = Conv2D(n_filters,
                  3,
                  activation='relu',
                  padding='same',
                  kernel_initializer='he_normal')(merge)
    
    conv = Conv2D(n_filters,
                  3,
                  activation='relu',
                  padding='same',
                  kernel_initializer='he_normal')(conv)
    
    return conv

In [16]:
# Unet Model

def unet_model(input_size=(256, 256, 3), n_filters=32):
    
    inputs = Input(input_size)
    
    # downsampling
    cblock1 = conv_block(inputs, n_filters)
    cblock2 = conv_block(cblock1[0], 2*n_filters)
    cblock3 = conv_block(cblock2[0], 4*n_filters)
    cblock4 = conv_block(cblock3[0], 8*n_filters, dropout_prob=0.3)
    
    # bottle-neck
    cblock5 = conv_block(cblock4[0], 16*n_filters, dropout_prob=0.3, max_pooling=False) 
    
    # upsampling
    ublock6 = upsampling_block(cblock5[0], cblock4[1],  8*n_filters)
    ublock7 = upsampling_block(ublock6, cblock3[1],  4*n_filters)
    ublock8 = upsampling_block(ublock7, cblock2[1],  2*n_filters)
    ublock9 = upsampling_block(ublock8, cblock1[1],  n_filters)

    conv9 = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(ublock9)

    conv10 = Conv2D(1, 1, padding='same', activation='sigmoid')(conv9)
    
    model = tf.keras.Model(inputs=inputs, outputs=conv10)

    return model

In [17]:
# Unet Model

def unet_model(input_size=(256, 256, 3), n_filters=32):
    
    inputs = Input(input_size)
    
    # downsampling
    cblock1 = conv_block(inputs, n_filters)
    cblock2 = conv_block(cblock1[0], 2*n_filters)
    cblock3 = conv_block(cblock2[0], 4*n_filters, dropout_prob=0.3)
    
    # bottle-neck
    cblock4 = conv_block(cblock3[0], 8*n_filters, dropout_prob=0.3, max_pooling=False) 
    
    # upsampling
    ublock5 = upsampling_block(cblock4[0], cblock3[1],  4*n_filters)
    ublock6 = upsampling_block(ublock5, cblock2[1],  2*n_filters)
    ublock7 = upsampling_block(ublock6, cblock1[1],  n_filters)

    conv8 = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(ublock7)

    conv9 = Conv2D(1, 1, padding='same', activation='sigmoid')(conv8)
    
    model = tf.keras.Model(inputs=inputs, outputs=conv9)

    return model

In [18]:
unet = unet_model((256, 256, 3))

In [19]:
unet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 9248        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 32) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [20]:
unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
# use early stopping to exit training if validation loss is not decreasing even after certain epochs (patience)
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# save the best model with least validation loss
checkpointer = ModelCheckpoint(filepath="localization_weights.h5", verbose=1, save_best_only=True)

In [22]:
STEP_SIZE_TRAIN = timage_generator.n/16
STEP_SIZE_VALID = vimage_generator.n/16

### Training and Testing

In [25]:
history = unet.fit(train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    batch_size=16,
                    epochs=100,
                    callbacks=[checkpointer, earlystopping],
                    validation_data=valid_gen,
                    validation_steps=STEP_SIZE_VALID)

Epoch 1/100
72/72 [==============================] - 62s 846ms/step - loss: 0.1054 - accuracy: 0.9700 - val_loss: 0.0978 - val_accuracy: 0.9733

Epoch 00001: val_loss improved from inf to 0.09783, saving model to localization_weights.h5
Epoch 2/100
72/72 [==============================] - 47s 644ms/step - loss: 0.1031 - accuracy: 0.9703 - val_loss: 0.0920 - val_accuracy: 0.9732

Epoch 00002: val_loss improved from 0.09783 to 0.09204, saving model to localization_weights.h5
Epoch 3/100
72/72 [==============================] - 47s 642ms/step - loss: 0.1052 - accuracy: 0.9701 - val_loss: 0.0951 - val_accuracy: 0.9725

Epoch 00003: val_loss did not improve from 0.09204
Epoch 4/100
72/72 [==============================] - 47s 642ms/step - loss: 0.1030 - accuracy: 0.9699 - val_loss: 0.0940 - val_accuracy: 0.9735

Epoch 00004: val_loss did not improve from 0.09204
Epoch 5/100
72/72 [==============================] - 47s 643ms/step - loss: 0.1016 - accuracy: 0.9702 - val_loss: 0.0919 - val_acc

In [27]:
# save the model architecture to json file for future use

model_json = unet.to_json()
with open("localization_model.json","w") as json_file:
    json_file.write(model_json)

In [26]:
unet.evaluate(test_gen, steps=testimage_generator.n/16, verbose=1)

6/6 [==============================] - 1s 190ms/step - loss: 0.0966 - accuracy: 0.9716


[0.09659004211425781, 0.9716346263885498]